Output:

*   A sample submission using simple Keras CNN
*   Predicted probability and truth labels for use in Best Threshold finding

To-do: Improve efficiency with multiprocessing from Blending Code

In [1]:

import numpy as np 
import pandas as pd 
import os
import gc

import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

import cv2
from tqdm import tqdm

from multiprocessing import Pool, cpu_count
from subprocess import check_output

In [2]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission_v2.csv to sample_submission_v2.csv
Saving test_v2_file_mapping.csv to test_v2_file_mapping.csv
Saving test-jpg-v2.tar to test-jpg-v2.tar
Saving train_v2.csv to train_v2.csv
Saving train-jpg.tar to train-jpg.tar


In [ ]:
x_train0 = []
x_test0 = []
y_train0 = []

df_train = pd.read_csv('train_v2.csv')

labels = df_train['tags'].str.get_dummies(sep=' ').columns

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

for f, tags in tqdm(df_train.values, miniters=1000):
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train0.append(cv2.resize(img, (32, 32)))
    y_train0.append(targets)
    
y_train0 = np.array(y_train0, np.uint8)
x_train0 = np.array(x_train0, np.float16) / 255.

print(x_train0.shape)
print(y_train0.shape)

100%|██████████| 40479/40479 [00:59<00:00, 680.48it/s]


(40479, 32, 32, 3)
(40479, 17)


In [ ]:
df_test = pd.read_csv('sample_submission_v2.csv')

for f, tags in tqdm(df_test.values, miniters=1000):
    img = cv2.imread('test-jpg-v2/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_test0.append(cv2.resize(img, (32, 32)))
    
x_test0 = np.array(x_test0, np.float16) / 255.

100%|██████████| 61191/61191 [01:32<00:00, 659.22it/s]


In [ ]:
split = 35000
x_train, x_valid, y_train, y_valid = x_train0[:split], x_train0[split:], y_train0[:split], y_train0[split:]

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32, 32, 3)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer='adam',
              metrics=['accuracy'])
              
model.fit(x_train, y_train,
          batch_size=128,
          epochs=3,
          verbose=1,
          validation_data=(x_valid, y_valid))
          
from sklearn.metrics import fbeta_score

p_train = model.predict(x_train0, batch_size=128,verbose=2)
p_test = model.predict(x_test0, batch_size=128,verbose=2)
# print(fbeta_score(y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))

Train on 35000 samples, validate on 5479 samples
Epoch 1/3
35000/35000 [==============================] - 227s - loss: 0.2500 - acc: 0.9045 - val_loss: 0.1994 - val_acc: 0.9206
Epoch 2/3
35000/35000 [==============================] - 229s - loss: 0.2040 - acc: 0.9204 - val_loss: 0.1807 - val_acc: 0.9278
Epoch 3/3
35000/35000 [==============================] - 231s - loss: 0.1910 - acc: 0.9259 - val_loss: 0.1777 - val_acc: 0.9289


In [ ]:
# Saving predicted probability and ground truth for Train Dataset
# Compute the best threshold externally
print(labels)
chk_output = pd.DataFrame()
for index in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]:
    chk_output['class %d' % index] = p_train[:,index-1]
chk_output.to_csv('predicted_probability.csv', index=False)
for index in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]:
    chk_output['class %d' % index] = y_train0[:,index-1]
chk_output.to_csv('true_label.csv', index=False)

Index(['agriculture', 'artisinal_mine', 'bare_ground', 'blooming', 'blow_down',
       'clear', 'cloudy', 'conventional_mine', 'cultivation', 'habitation',
       'haze', 'partly_cloudy', 'primary', 'road', 'selective_logging',
       'slash_burn', 'water'],
      dtype='object')


In [ ]:
values_test = (p_test > .222222)*1.0        # before multiplying by 1.0, this appears as an array of True and False
values_test = np.array(values_test, np.uint8)

print(values_test)
# Build Submission, using label outputted from long time ago
test_labels = []
for row in range(values_test.shape[0]):
    test_labels.append(' '.join(labels[values_test[row,:]==1]))
Submission_PDFModel = df_test.copy()
Submission_PDFModel.drop('tags', axis = 1)
Submission_PDFModel['tags'] = test_labels
Submission_PDFModel.to_csv('submission_hamoye.csv', index = False)

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 1]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 1]]
